<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [2]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [3]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [4]:
df = df.sample(False, 0.5)

In [5]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [6]:
df = df.fillna(0, subset=num_columns)

In [8]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model_MTE'))

In [9]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1832852

In [10]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [11]:
!pip3.5 install hyperopt

     |████████████████████████████████| 1.9 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 59 kB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 614 kB/s eta 0:00:01
    Running setup.py install for networkx ... done
    Running setup.py install for future ... done


## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [35]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [36]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [14]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [15]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.725627934940136

In [16]:
all_metrics = {}

In [17]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [ ]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

In [19]:
best

{'eta': 0, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [21]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

In [25]:
eta, num_round

(0.5, 100)

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [48]:
max_depth_choice = [3, 4, 5, 6, 8, 10]
min_child_weight_choice = [1., 3., 5., 7.]

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    
    # Fixed    
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    'num_round': num_round,
    'eta': eta,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 0.9, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 3, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 5.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5041161049531627, ROC-AUC: 0.7332839384033639
SPACE:                                                                          
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 0.9, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 10, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 5.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5055472624270787, ROC-AUC: 0.7318608047939822                       
SPACE:                                                                          
{'eval_metric': 'logloss', 

LOG-LOSS: 0.5010966269935865, ROC-AUC: 0.7374961444269488                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 0.9, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 3, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5033106367801168, ROC-AUC: 0.7343945733346177                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 0.9, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 5, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 5.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5019185259541359, ROC-AUC: 0.73618423572

In [49]:
max_depth = max_depth_choice[best['max_depth']] 
min_child_weight = min_child_weight_choice[best['min_child_weight']] 

In [50]:
max_depth, min_child_weight

(6, 3.0)

#### Tune gamma

In [53]:
gamma_choice = [3., 4., 5., 6., 8., 10.]

space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),
    
    # Fixed    
    'subsample': baseline_params['subsample'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    'num_round': num_round,
    'eta': eta,
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5005293964635392, ROC-AUC: 0.7380299611125478
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5003470672616955, ROC-AUC: 0.7384995442259997                        
SPACE:                                                                           
{'eval_metric': 'logloss'

LOG-LOSS: 0.5000752351752623, ROC-AUC: 0.7390350289456487                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5003470672616955, ROC-AUC: 0.7384995442260011                         
SPACE:                                                                            
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 6.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.9, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.9}
LOG-LOSS: 0.5009899875883698, ROC-AUC: 0.73754098893

In [54]:
gamma = gamma_choice[best['gamma']]

#### Tune subsample, colsample_bytree

In [58]:
subsample_choice = [0.3, 0.5 , 0.7, 1.0]
colsample_bytree_choice = [0.3, 0.4, 0.5 , 0.7]

space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed    
    
    'num_round': num_round,
    'eta': eta,
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    'gamma': gamma,
    
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.7, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.3}
LOG-LOSS: 0.5025602173681574, ROC-AUC: 0.7357319554397082
SPACE:                                                                          
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.5, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.3}
LOG-LOSS: 0.5023020947512918, ROC-AUC: 0.7360994545302626                       
SPACE:                                                                          
{'eval_metric': 'logloss', '

LOG-LOSS: 0.5011867955355729, ROC-AUC: 0.7371731031692652                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.7, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 0.5}
LOG-LOSS: 0.5015032327270996, ROC-AUC: 0.7369981220190694                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'colsample_bytree': 0.3, 'max_depth': 6, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'subsample': 1.0}
LOG-LOSS: 0.5014236661386957, ROC-AUC: 0.736995191203536

In [59]:
subsample = subsample_choice[best['subsample']]
colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [61]:
reg_lambda_choice = [0., 0.01, 0.1, 1., 3.]
alpha_choice = [0., 0.01, 0.1, 1., 3.]

space = {
    # alpha
    'alpha': hp.choice('alpha', alpha_choice),
    'reg_lambda': hp.choice('reg_lambda', reg_lambda_choice),
    
    # Fixed    
    
    'num_round': num_round,
    'eta': eta,
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'lambda': 3.0, 'colsample_bytree': 0.5, 'alpha': 0.0, 'subsample': 1.0, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'max_depth': 6}
LOG-LOSS: 0.5006739883142302, ROC-AUC: 0.7380464340087293
SPACE:                                                                          
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'lambda': 1.0, 'colsample_bytree': 0.5, 'alpha': 3.0, 'subsample': 1.0, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'max_depth': 6}
LOG-LOSS: 0.5004435368627966, ROC-AUC: 0.7382430740853558                       
SPACE:                                             

LOG-LOSS: 0.5008778673828328, ROC-AUC: 0.7378295327685568                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'lambda': 0.01, 'colsample_bytree': 0.5, 'alpha': 1.0, 'subsample': 1.0, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'max_depth': 6}
LOG-LOSS: 0.5004023270660688, ROC-AUC: 0.73852161703887                          
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'gamma': 8.0, 'num_round': 100, 'nthread': 1, 'lambda': 3.0, 'colsample_bytree': 0.5, 'alpha': 0.0, 'subsample': 1.0, 'featuresCol': 'features', 'objective': 'binary:logistic', 'predictionCol': 'prediction', 'eta': 0.5, 'nworkers': 4, 'min_child_weight': 3.0, 'silent': 0, 'max_depth': 

In [62]:
alpha = alpha_choice[best['alpha']]
reg_lambda = reg_lambda_choice[best['reg_lambda']]

In [63]:
alpha,reg_lambda

(0.01, 0.0)

### Lower the learning rate and decide the optimal parameters

In [66]:
space = {
    
    'alpha': alpha, 
    'reg_lambda': reg_lambda,
    'num_round': num_round,
    'eta': eta,
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    
    **static_params
}


estimator_xgb = XGBoostEstimator(**space)
model_xgb = estimator_xgb.fit(train_df)

---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [69]:
from pyspark.ml.classification import LogisticRegression

In [74]:
def objective(space):
    estimator = LogisticRegression(**space)
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probability')
    roc_auc = rocauc(model, val_df, probabilities_col='probability')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [77]:
maxIter_choice = [50, 100, 30]
regParam_choice = [0.0, 0.01, 0.1, 1.0]
elasticNetParam_choice = [0.0, 0.01, 0.1, 0.3, 1.0]

space = {
    'maxIter': hp.choice('maxIter', max_iter_choice),
    'regParam': hp.choice('regParam', reg_param_choice),
    'elasticNetParam': hp.choice('elasticNetParam', elastic_net_param_choice)
}

In [78]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=36,
    trials=trials
)

LOG-LOSS: 0.5571442610423667, ROC-AUC: 0.6989147257387636
LOG-LOSS: 0.5746449703432287, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5312851113935829, ROC-AUC: 0.7015632803530526                       
LOG-LOSS: 0.5293335042413531, ROC-AUC: 0.7015664518378846                       
LOG-LOSS: 0.5353396802035896, ROC-AUC: 0.6999577847856365                       
LOG-LOSS: 0.5312851108129653, ROC-AUC: 0.7015632804301422                       
LOG-LOSS: 0.5746449703432287, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5583010275976711, ROC-AUC: 0.6745949385680562                       
LOG-LOSS: 0.5337866944299171, ROC-AUC: 0.6987269972099683                       
LOG-LOSS: 0.5337866944299171, ROC-AUC: 0.6987269972099659                       
LOG-LOSS: 0.5312851115442675, ROC-AUC: 0.7015632803530514                        
LOG-LOSS: 0.5571442610423667, ROC-AUC: 0.6989147257387659                        
LOG-LOSS: 0.5293335042413531, ROC-AUC: 0.70156645

In [79]:
maxIter = maxIter_choice[best['maxIter']]
regParam = regParam_choice[best['regParam']]
elasticNetParam = elasticNetParam_choice[best['elasticNetParam']]

In [80]:
logreg = LogisticRegression(maxIter=maxIter, regParam=regParam, elasticNetParam=elasticNetParam)
model_logreg = logreg.fit(train_df)

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [82]:
logreg_metrics = {
    'logloss': logloss(model_logreg, test_df, probabilities_col='probability'),
    'rocauc': rocauc(model_logreg, test_df, probabilities_col='probability')
}

all_metrics['logreg_metrics'] = logreg_metrics

In [83]:
xgb_metrics = {
    'logloss': logloss(model_xgb, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(model_xgb, test_df, probabilities_col='probabilities')
}

all_metrics['xgb_metrics'] = xgb_metrics

In [84]:
all_metrics

{'logreg_metrics': {'logloss': 0.5308505896357181,
  'rocauc': 0.6991716617166996},
 'xgb baseline': {'logloss': 0.5145790867988863, 'rocauc': 0.7237139054629188},
 'xgb_metrics': {'logloss': 0.504730755829568, 'rocauc': 0.7365324328873123}}

Итоговая таблица

In [86]:
get_ate(all_metrics, CONTROL_NAME)

,logreg_metrics ate %,metric,xgb_metrics ate %
0,3.162100,logloss,-1.913861
1,-3.391153,rocauc,1.771215


In [ ]:
eta=0.5, num_round=100, max_depth=6, min_child_weight=3.0, 
gamma=8.0, alpha=0.01, reg_lambda=0.0, subsample=1.0, colsample_bytree=0.5

In [102]:
static_params={'eval_metric': 'logloss',
 'featuresCol': 'features',
 'labelCol': 'label',
 'nthread': 1,
 'nworkers': 4,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'silent': 0}

In [ ]:
space = {
    
    'alpha': alpha, 
    'reg_lambda': reg_lambda,
    'num_round': num_round,
    'eta': eta,
    'min_child_weight': min_child_weight,
    'max_depth': max_depth,
    'gamma': gamma,
    'subsample': subsample,
    'colsample_bytree': colsample_bytree,
    
    
    **static_params
}

In [87]:
DATA_PATH = '/workspace/data/criteo'

TEST_PATH = os.path.join(DATA_PATH, 'test.csv')

TEST = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TEST_PATH)

TEST = TEST\
    .fillna(0, subset=num_columns)

In [89]:
TEST_trans = pipeline_model \
    .transform(TEST) \
    .select('features', 'id') \
    .cache()

In [95]:
pred_TEST = model_xgb.transform(TEST_trans)

In [96]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

second_el = udf(lambda v: float(v[1]), FloatType())

In [99]:
result = pred_TEST\
    .select('id', second_el('probabilities').alias('proba'))

In [103]:
result.sort('id').toPandas().to_csv('./answers.csv', index=None)